In [1]:
import pandas as pd
import numpy as np
import datetime
import ast
import warnings
from ast import literal_eval
import string
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import StandardScaler
from scipy.sparse import coo_matrix, csr_matrix, save_npz, load_npz, hstack
import random
from sklearn.metrics import mean_squared_error

favorites = ['The Count of Monte Cristo (2002)',"Lord of the Rings: The Return of the King, The (2003)",'The Imitation Game (2014)']
non_favorites = ['Around the World in 80 Days (2004)', 'Gentlemen of Fortune (Dzhentlmeny udachi) (1972)'
                 , 'Friday the 13th Part 3: 3D (1982)', 'Swing Vote (2008)', 'Fish Called Wanda, A (1988)'
                 , 'Dark Blue (2003)', 'We Bought a Zoo (2011)', 'Mouchette (1967)', 'My Favorite Wife (1940)'
                 , '24 Hour Party People (2002)', 'American Me (1992)', 'Your Friends and Neighbors (1998)'
                 , 'Amarcord (1973)', 'Bob le Flambeur (1955)', 'Songs From the Second Floor (Sånger från andra våningen) (2000)'
                 , 'Bliss (1997)', 'Night of the Creeps (1986)', 'Best Exotic Marigold Hotel, The (2011)', 'Pandorum (2009)'
                ,"Cheech & Chong's The Corsican Brothers (1984)", 'Great Buck Howard, The (2008)', 'Hamburger Hill (1987)', 
                 'Head of State (2003)', 'Morvern Callar (2002)', 'Harvard Man (2001)', "Relax... It's Just Sex (1998)"
                 , 'Zeus and Roxanne (1997)', 'Poseidon Adventure, The (1972)', 'Here Comes the Boom (2012)']

In [2]:
moviesdf = pd.read_csv(r'C:\Users\marsh\Desktop\Capstone 3\website\moviesdf4.csv')
ratingsdf = pd.read_csv('ratingsdf3.csv')


,title,poster_url
0,"""Great Performances"" Cats (1998)",https://image.tmdb.org/t/p/w200//yqJ8aU0nooUCH...
1,'Round Midnight (1986),https://image.tmdb.org/t/p/w200//8aNGnEsvLBldN...
2,'Salem's Lot (2004),https://image.tmdb.org/t/p/w200/hcdFVK9CLsnHIF...
3,'Til There Was You (1997),https://image.tmdb.org/t/p/w200//ndbW89bKquf5g...
4,"'burbs, The (1989)",https://image.tmdb.org/t/p/w200//vrVPAcv2njVdn...
...,...,...
9138,eXistenZ (1999),https://image.tmdb.org/t/p/w200//kETKF0JhdTPn1...
9139,xXx (2002),https://image.tmdb.org/t/p/w200//86BnliVDBo2Tj...
9140,xXx: State of the Union (2005),https://image.tmdb.org/t/p/w200//MM1r8HQUn6VG0...
9141,¡Three Amigos! (1986),https://image.tmdb.org/t/p/w200//keQ4ra50w1nkt...


In [4]:
pivot = ratingsdf.pivot(index = 'title',columns = 'userId', values = 'rating')
pivot

userId,1,2,3,5,7,8,11,13,14,16,...,138474,138475,138477,138483,138484,138486,138487,138490,138492,138493
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"'burbs, The (1989)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xXx (2002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xXx: State of the Union (2005),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
train_data, test_data = train_test_split(pivot, test_size = 0.2)

In [29]:
# Normalize movies by dividing by sqrt of # of ratings
countstest = np.sqrt(test_data.count(axis = 1))
counts = np.sqrt(train_data.count(axis = 1)) 
test_norm = test_data.div(countstest,axis = 0)
train_norm = train_data.div(counts, axis = 0)
train_norm = train_norm.fillna(0)
test_norm = test_norm.fillna(0)
test_norm = test_norm.values
train_norm = train_norm.values
num_latent_factors = 50
U, s, Vt = svds(train_norm, k=num_latent_factors)
s_diag = np.diag(s)
predicted_ratings = np.dot(np.dot(U, s_diag), Vt)

# Evaluate the model on the test set
nonzero_indices = test_norm.nonzero()
test_ratings = test_norm[nonzero_indices]
test_row_indices = nonzero_indices[0]
test_col_indices = nonzero_indices[1]
predicted_test_ratings = predicted_ratings[test_row_indices, test_col_indices]
rmse = np.sqrt(mean_squared_error(test_ratings, predicted_test_ratings))
mse = mean_squared_error(test_ratings, predicted_test_ratings)
print(f"RMSE: {rmse:.3f}, MSE: {mse:.3f}")

RMSE: 0.074, MSE: 0.005


In [ ]:
# Normalize movies by dividing by # of ratings
countstest = test_data.count(axis = 1)
counts = train_data.count(axis = 1)
test_norm = test_data.div(countstest,axis = 0)
train_norm = train_data.div(counts, axis = 0)
train_norm = train_norm.fillna(0)
test_norm = test_norm.fillna(0)
test_norm = test_norm.values
train_norm = train_norm.values
num_latent_factors = 50
U, s, Vt = svds(train_norm, k=num_latent_factors)
s_diag = np.diag(s)
predicted_ratings = np.dot(np.dot(U, s_diag), Vt)

# Evaluate the model on the test set
nonzero_indices = test_norm.nonzero()
test_ratings = test_norm[nonzero_indices]
test_row_indices = nonzero_indices[0]
test_col_indices = nonzero_indices[1]
predicted_test_ratings = predicted_ratings[test_row_indices, test_col_indices]
rmse = np.sqrt(mean_squared_error(test_ratings, predicted_test_ratings))
mse = mean_squared_error(test_ratings, predicted_test_ratings)
print(f"RMSE: {rmse:.3f}, MSE: {mse:.3f}")

In [18]:
# Recommendations for division by # of ratings

newdf = pd.DataFrame(index = moviesdf.title)
newdf['new_user_rating'] = np.nan
newdf.loc[favorites,'new_user_rating'] = 5
newdf.loc[non_favorites,'new_user_rating'] = 1
new = newdf.values
counts = pivot.count(axis = 1)
norm = pivot.div(counts, axis = 0)
old = norm.values
ui_new = np.hstack((old,new))
ui_new = np.nan_to_num(ui_new, nan = 0)
U,S, Vt = svds(ui_new, k=50)
Uk = U
Sk = np.diag(S)
Vk = Vt.T
    # Compute the predicted ratings for the new user using the truncated SVD
new_user_ratings = Uk @ Sk @ Vk[-1,:].T
rec = pd.DataFrame(data= new_user_ratings, columns = ['rating'])
rec['title'] = moviesdf.title
rec = rec[~rec.title.isin(favorites)]
rec = rec.sort_values('rating', ascending = False).head(10)
rec = rec.merge(moviesdf, on = 'title',how ='inner')
rec

,rating,title,poster_url
0,1.001589,Gentlemen of Fortune (Dzhentlmeny udachi) (1972),https://image.tmdb.org/t/p/w200//owZ2KQ2x99XqM...
1,1.001140,Here Comes the Boom (2012),https://image.tmdb.org/t/p/w200//g3ZjNkQP4M7Cs...
2,1.001087,Mouchette (1967),https://image.tmdb.org/t/p/w200//raizDhYcnFXwI...
3,1.001060,"Great Buck Howard, The (2008)",https://image.tmdb.org/t/p/w200//9x5EajLkxeS3v...
4,1.000745,Swing Vote (2008),https://image.tmdb.org/t/p/w200//590NPZbaL63cD...
5,1.000723,Relax... It's Just Sex (1998),https://image.tmdb.org/t/p/w200//7IfXHRstu6TRy...
6,1.000685,American Me (1992),https://image.tmdb.org/t/p/w200//eeoK2gRfAxZyY...
7,1.000514,We Bought a Zoo (2011),https://image.tmdb.org/t/p/w200//dcOvIqdsojUdA...
8,1.000498,Morvern Callar (2002),https://image.tmdb.org/t/p/w200//qcX86XKhPH4Q0...
9,1.000488,Bob le Flambeur (1955),https://image.tmdb.org/t/p/w200//3lhgRzQJif5UA...


In [5]:
# Recommendations for division by sqrt of # of ratings

newdf = pd.DataFrame(index = moviesdf.title)
newdf['new_user_rating'] = np.nan
newdf.loc[favorites,'new_user_rating'] = 5
newdf.loc[non_favorites,'new_user_rating'] = 1
new = newdf.values
counts = np.sqrt(pivot.count(axis = 1))
norm = pivot.div(counts, axis = 0)
old = norm.values
ui_new = np.hstack((old,new))
ui_new = np.nan_to_num(ui_new, nan = 0)
U,S, Vt = svds(ui_new, k=50)
Uk = U
Sk = np.diag(S)
Vk = Vt.T
    # Compute the predicted ratings for the new user using the truncated SVD
new_user_ratings = Uk @ Sk @ Vk[-1,:].T
rec = pd.DataFrame(data= new_user_ratings, columns = ['rating'])
rec['title'] = moviesdf.title
rec = rec[~rec.title.isin(favorites)]
rec = rec.sort_values('rating', ascending = False).head(10)
rec = rec.merge(moviesdf, on = 'title',how ='inner')
rec

,rating,title,poster_url
0,0.958115,Gentlemen of Fortune (Dzhentlmeny udachi) (1972),https://image.tmdb.org/t/p/w200//owZ2KQ2x99XqM...
1,0.945106,24 Hour Party People (2002),https://image.tmdb.org/t/p/w200//8f50NQ1wz6hdx...
2,0.941127,"Great Buck Howard, The (2008)",https://image.tmdb.org/t/p/w200//9x5EajLkxeS3v...
3,0.930001,My Favorite Wife (1940),https://image.tmdb.org/t/p/w200//eRyGZiEWGn9bA...
4,0.926723,Amarcord (1973),https://image.tmdb.org/t/p/w200//s6e5B6ty6xjLU...
5,0.918453,Songs From the Second Floor (Sånger från andra...,https://image.tmdb.org/t/p/w200//heW06kNneH6nL...
6,0.905769,American Me (1992),https://image.tmdb.org/t/p/w200//eeoK2gRfAxZyY...
7,0.903482,Swing Vote (2008),https://image.tmdb.org/t/p/w200//590NPZbaL63cD...
8,0.900060,Bob le Flambeur (1955),https://image.tmdb.org/t/p/w200//3lhgRzQJif5UA...
9,0.898750,Dark Blue (2003),https://image.tmdb.org/t/p/w200//9RTiJx3oD1s3R...
